In [ ]:
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/data_nlp'
PRED_PATH = os.path.join(BASE_DIR, 'predictions.csv')

Mounted at /content/drive


Conncect with google drive and Importing the paths

install some packages

In [ ]:
!pip install --quiet spacy networkx scikit-learn joblib
!python -m spacy download en_core_web_sm
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 30.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.7 MB/s eta 0:00:00


Importing the libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import networkx as nx
import re
import joblib
import torch

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from transformers import RobertaTokenizer, RobertaModel

Load the dataset

In [ ]:
abstracts = pd.read_csv(f'{BASE_DIR}/abstracts.txt', sep=r'\|\-\-\|', engine='python', names=['id', 'text'], header=None)
authors   = pd.read_csv(f'{BASE_DIR}/authors.txt',   sep=r'\|\-\-\|', engine='python', names=['id', 'auth'], header=None)
edgelist  = pd.read_csv(f'{BASE_DIR}/edgelist.txt', names=['src', 'tgt'], header=None)

meta = abstracts.merge(authors, on='id').reset_index(drop=True)
all_ids = meta['id'].values
id2idx = {}
for i, pid in enumerate(all_ids):
    id2idx[pid] = i

Graph creation for identifying edges between papers

In [ ]:
all_ids = meta['id'].values
id2idx = {pid:i for i,pid in enumerate(all_ids)}
G = nx.Graph()
G.add_edges_from(edgelist.values)
neighbors = {n:set(G.neighbors(n)) for n in G.nodes()}
degrees = {nid:G.degree(nid) for nid in all_ids}

precompute rage rank and clustering

In [ ]:
pagerank = nx.pagerank(G)
clustering = nx.clustering(G)
pr_array = np.array([pagerank.get(pid, 0) for pid in all_ids], dtype=float)
clust_array = np.array([clustering.get(pid, 0) for pid in all_ids], dtype=float)

Text preprocessing

In [ ]:
raw_texts = meta['text'].fillna('').tolist()
proc_texts = [" ".join(re.findall(r"\b[a-zA-Z]{3,}\b", t.lower())) for t in raw_texts]

tfidf

In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_tfidf = tfidf.fit_transform(proc_texts).toarray()
# Precompute top-k token sets for Jaccard
k = 50
topk_tokens = []
for vec in X_tfidf:
    top_idx = np.argsort(vec)[-k:]
    topk_tokens.append(set(top_idx))

Embeddings with SentenceTransformer()

In [ ]:
model_embed = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
X_doc_vecs = model_embed.encode(proc_texts, show_progress_bar=True, batch_size=64, convert_to_numpy=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/2165 [00:00<?, ?it/s]

roBERTa tokenization

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model_roberta = RobertaModel.from_pretrained('roberta-base').to('cuda')
model_roberta.eval()

# tokenize and encoding in only one batch

batch_size = 16  # ή ακόμα και 8
embeddings = []
for i in range(0, len(proc_texts), batch_size):
    batch = proc_texts[i:i+batch_size]
    enc = tokenizer(batch, padding='longest', truncation=True, max_length=128, return_tensors='pt').to('cuda')
    with torch.no_grad():
        out = model_roberta(**enc)
    embeddings.append(out.last_hidden_state[:, 0, :].cpu().numpy())
    torch.cuda.empty_cache()

X_doc_vecs_roberta = np.vstack(embeddings)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Author sets for Jaccard Similarity

In [ ]:
auth_sets = meta['auth'].fillna('').str.split(',').map(lambda L: {a.strip() for a in L}).tolist()


Function for JaccardSimilarity

In [ ]:
def jaccard(a, b):
    return float(len(a & b)) / len(a | b) if (a | b) else 0.0

Negative/Positive pair creation

In [ ]:
pos = edgelist.values.tolist()
rng = np.random.default_rng(42)
neg = []

while len(neg) < len(pos):
    u, v = rng.choice(all_ids, 2, replace=False)
    if not G.has_edge(u, v):
        neg.append([u, v])

pairs = np.array(pos + neg)
labels = np.array([1] * len(pos) + [0] * len(neg))

Feature Extraction

In [ ]:
features = []
for u, v in pairs:
    ui, vi = id2idx[u], id2idx[v]
    # semantic embeddings
    vec_u, vec_v = X_doc_vecs[ui], X_doc_vecs[vi]
    sim = np.dot(vec_u, vec_v)
    dist = np.linalg.norm(vec_u - vec_v)

    #roberta embeddings
    vec_u1, vec_v1 = X_doc_vecs_roberta[ui], X_doc_vecs_roberta[vi]
    sim_roberta = np.dot(vec_u1, vec_v1)
    cos_roberta = sim_roberta / (
        (np.linalg.norm(vec_u1) * np.linalg.norm(vec_v1)) + 1e-8
    )
    dist_roberta = np.linalg.norm(vec_u1 - vec_v1)

    # TF-IDF features
    tfidf_sim = np.dot(X_tfidf[ui], X_tfidf[vi]) / (
        np.linalg.norm(X_tfidf[ui]) * np.linalg.norm(X_tfidf[vi]) + 1e-8)
    jacc_tfidf = jaccard(topk_tokens[ui], topk_tokens[vi])

    # authors
    auth_sim = jaccard(auth_sets[ui], auth_sets[vi])

    # graph features
    deg_diff = abs(degrees[ui] - degrees[vi])
    cn = len(neighbors[u] & neighbors[v])
    aa_score = sum(1 / np.log(degrees[id2idx[w]]) for w in (neighbors[u] & neighbors[v]) if degrees[id2idx[w]] > 1)
    pr_diff = abs(pr_array[ui] - pr_array[vi])
    clust_diff = abs(clust_array[ui] - clust_array[vi])

    # token lengths
    tok_diff = abs(len(proc_texts[ui].split()) - len(proc_texts[vi].split()))
    features.append([sim, dist, sim_roberta, cos_roberta, dist_roberta, tfidf_sim, jacc_tfidf, auth_sim,
                     deg_diff, cn, aa_score, pr_diff, clust_diff, tok_diff])
X = np.array(features)


scaling the features

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

train test split

In [ ]:
X_tr, X_val, y_tr, y_val = train_test_split(X, labels, test_size=0.1, stratify=labels, random_state=42)


XGBoostmodel training

In [ ]:
xg_model = XGBClassifier(
    objective='binary:logistic', n_jobs=-1, random_state=42,
    max_depth=7, learning_rate=0.05, n_estimators=400,
    subsample=0.8, colsample_bytree=0.8, scale_pos_weight=1
)
xg_model.fit(X_tr, y_tr)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=400, n_jobs=-1,
              num_parallel_tree=None, random_state=42, ...)

catboost training

In [ ]:
# catboost_params = {
#     'iterations': 300,
#     'learning_rate': 0.01,
#     'eval_metric': 'Accuracy',
#     'task_type': 'GPU',
#     'early_stopping_rounds': 20,
#     'verbose': 50
# }

cat_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    od_type='Iter',
    od_wait=50,
    task_type='GPU',
    verbose=100
)
cat_model.fit(X_tr, y_tr)

0:	learn: 0.9515207	total: 312ms	remaining: 5m 11s
100:	learn: 0.9594072	total: 3.67s	remaining: 32.7s
200:	learn: 0.9599292	total: 8.6s	remaining: 34.2s
300:	learn: 0.9601393	total: 11.9s	remaining: 27.6s
400:	learn: 0.9603072	total: 15.1s	remaining: 22.6s
500:	learn: 0.9604598	total: 20.1s	remaining: 20s
600:	learn: 0.9605962	total: 23.3s	remaining: 15.5s
700:	learn: 0.9607035	total: 26.5s	remaining: 11.3s
800:	learn: 0.9608236	total: 29.7s	remaining: 7.38s
900:	learn: 0.9609518	total: 34.6s	remaining: 3.8s
999:	learn: 0.9610418	total: 37.8s	remaining: 0us


svm

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_tr, y_tr)



evaluation

In [ ]:

val_preds_xg = xg_model.predict_proba(X_val)[:, 1]
val_preds_cat = cat_model.predict_proba(X_val)[:, 1]
final_probs_AUC = (xg_preds + cat_preds) / 2
print("Validation AUC:", roc_auc_score(y_val, final_probs_AUC))
# val_preds_svm = svm_model.predict_proba(X_val)[:, 1]
# print("validation AUC:", roc_auc_score(y_val, val_preds_svm))


test prediction

In [ ]:
pairs_test = pd.read_csv(os.path.join(BASE_DIR, 'test.txt'), names=['src', 'tgt']).values
# extract test features using same logic
features_test = []
for u, v in pairs_test:

    #semantic embeddings
    ui, vi = id2idx[u], id2idx[v]
    vec_u, vec_v = X_doc_vecs[ui], X_doc_vecs[vi]
    sim = np.dot(vec_u, vec_v)
    dist = np.linalg.norm(vec_u - vec_v)

    #roberta embeddings
    vec_u1, vec_v1 = X_doc_vecs_roberta[ui], X_doc_vecs_roberta[vi]
    sim_roberta = np.dot(vec_u1, vec_v1)
    cos_roberta = sim_roberta / (
        (np.linalg.norm(vec_u1) * np.linalg.norm(vec_v1)) + 1e-8
    )
    dist_roberta = np.linalg.norm(vec_u1 - vec_v1)

    #tf-idf features
    tfidf_sim = np.dot(X_tfidf[ui], X_tfidf[vi]) / (
        np.linalg.norm(X_tfidf[ui]) * np.linalg.norm(X_tfidf[vi]) + 1e-8)
    jacc_tfidf = jaccard(topk_tokens[ui], topk_tokens[vi])

    #authors
    auth_sim = jaccard(auth_sets[ui], auth_sets[vi])

    # graph features
    deg_diff = abs(degrees[ui] - degrees[vi])
    cn = len(neighbors[u] & neighbors[v])
    aa_score = sum(1 / np.log(degrees[id2idx[w]]) for w in (neighbors[u] & neighbors[v]) if degrees[id2idx[w]] > 1)
    pr_diff = abs(pr_array[ui] - pr_array[vi])
    clust_diff = abs(clust_array[ui] - clust_array[vi])

    # token lengths
    tok_diff = abs(len(proc_texts[ui].split()) - len(proc_texts[vi].split()))
    features_test.append([sim, dist, sim_roberta, cos_roberta, dist_roberta, tfidf_sim, jacc_tfidf, auth_sim,
                          deg_diff, cn, aa_score, pr_diff, clust_diff, tok_diff])

# normalization
X_test = scaler.transform(np.array(features_test))

xg_preds = xg_model.predict_proba(X_test)[:, 1]
cat_preds = cat_model.predict_proba(X_test)[:, 1]

final_probs = (xg_preds + cat_preds) / 2
# final_probs = svm_model.predict_proba(X_test)[:, 1]

Model prediction and save .csv file

In [ ]:
pd.DataFrame({'ID': range(len(final_probs)), 'label': final_probs}).to_csv(PRED_PATH, index=False, float_format='%.6f')
